# LAB 03: SLIDING PUZZLE

## Imports

In [4]:
from collections import namedtuple
from heapq import heappush, heappop
from random import choice
from tqdm.auto import tqdm
import numpy as np

## Utility functions

In [ ]:
def count_inversions(board):
        """
        Count the number of inversions in the board.
        Flatten the board and ignore the blank (0) tile.
        :Parameters:
        board (np.ndarray): the game board
        :Returns: inversions (int): number of inversions
        """
        flat_board = board.flatten()
        flat_board = flat_board[flat_board != 0]  # Remove the blank (0) tile
        inversions = 0
        for i in range(len(flat_board)):
            for j in range(i + 1, len(flat_board)):
                if flat_board[i] > flat_board[j]:
                    inversions += 1
        return inversions


def is_solvable(board):
    """
    Check if the puzzle is solvable using inversion count and blank row position.
    :Parameters:
    board: (np.ndarray): the game board
    :Returns: bool
    """
    inversions = count_inversions(board)
    if board.size % 2 == 1:  # Odd-sized grid
        return inversions % 2 == 0
    else:  # Even-sized grid
        blank_row = board.size - np.where(board == 0)[0][0]  # Row counting from bottom
        return (inversions + blank_row) % 2 == 0


def generate_board(n):
    '''
    Generates a random play board of dimension n
    :Parameters: n (int): the dimension of the board
    :Returns: board (np.ndarray): the play board as square matrix of dimension n
    '''
    
    tiles = np.arange(n**2,dtype=int)
    np.random.shuffle(tiles)
    board = np.reshape(tiles,((n,n)))

    while not is_solvable(board):
        np.random.shuffle(tiles)
    

    return board


def print_board(board, return_state = False):
    """
    Print the current board state and returns the game state as string if specified
    :Paremeters:
    \tboard (np.ndarray): the game board\n
    \treturn_state (bool, optional, default value: False): set if you want to get the game state as a string\n
    :Returns:
    state (str, optional): the game state
    """

    state = '\n'.join([' '.join(map(str, row)) for row in board])
    print(state)
    if return_state:
        return state
    
def is_valid_move(pos:tuple, dimensions:int):
    '''Checks if a position is valid within the puzzle boundaries.'''
    return pos[0]<dimensions and pos[0]>=0 and pos[1]<dimensions and pos[1]>=0


def swap(board, pos1, pos2):
    '''Swaps two tiles in the puzzle.'''
    new_board=np.copy(board)
    tmp = new_board[pos1[0]][pos1[1]]
    new_board[pos1[0]][pos1[1]] = new_board[pos2[0]][pos2[1]]
    new_board[pos2[0]][pos2[1]] = tmp
    return new_board

def node_already_present(graph,state):
    
    for node in graph:
        if np.all(node.current_state==state):
            return True
        
    return False

## Proposed Solutions

### BFS

In [ ]:
class Node:

    def __init__(self,parent_state,current_state,depth):
        self.parent_state=parent_state
        self.current_state=current_state
        self.depth = depth
        
    def display(self):
        print_board(self.current_state)

def BFS(start_state:np.ndarray, goal:np.ndarray, start = 0):
    '''
    Function for the BFS uniniformed strategy for the Sliding puzzle
    :Parameters:
    \tboard (np.ndarray): The game board\n
    \tgoal (np.ndarray): The board goal configuration\n
    \tstart (int, optional, default = 0): The starting point representing the empty tile, defaultly coded with 0\n
    :Returns:
    \tmoves: The moves necessary to reach the goal configuration
    '''

    if not is_solvable(start_state) and start_state.shape[0]!=2:
        #cases not provided by the 'generate_board()' function must be tested
        return('This board cannot be solved!')

    directions =[
                    (-1,0), #left
                    (1,0),  #right
                    (0,-1), #down
                    (0,1),  #right
                ]
    
    root = Node(None,start_state,0)
    graph = [root]      #this is the graph in its entrirety
    frontier = [root]   #this represent the frontier from which BFS extract the node to continue BFS
    solution_found=False

    while not solution_found and len(frontier)!=0:

        new_frontier=[]
        
        while len(frontier)!=0 :

            starting_node=frontier.pop()
            starting_board=starting_node.current_state
            start_coordinates=np.argwhere(starting_board==start)[0]
            start_tile=(start_coordinates[0],start_coordinates[1])

            for direction in directions:

                swapping_tile = tuple(map(lambda x, y: x + y, start_tile, direction))

                if not is_valid_move(swapping_tile,start_state.shape[0]):
                    continue

                landing_board=swap(starting_board,start_tile,swapping_tile)

                if node_already_present(graph,landing_board):
                    #check if this step has been already considered
                    continue

                new_node = Node(starting_node,landing_board,starting_node.depth+1)
                graph.append(new_node)
                
                
                if np.all(goal==landing_board):
                    solution_found=True
                    solution_node=new_node

                new_frontier.append(new_node)

        frontier=new_frontier

    if solution_found:
        current_node=solution_node

        path=[]
        
        while current_node is not None:
            path.append(current_node)
            current_node=current_node.parent_state
        
        print(f'Solution found after {solution_node.depth} moves')
        
        for state in path:
            state.display()
            print('-->')
        print('Done!')
    
    else:

        print('Cannot find solution for this board')
        for node in graph:

            node.display()
            print("------------------")


### DFS

In [140]:
def DFS_R(graph:list,path:list, start_state:Node, goal:np.ndarray, start=0):
    if not is_solvable(start_state.current_state) and start_state.current_state.shape[0]!=2:
        #cases not provided by the 'generate_board()' function must be tested
        return False

    if np.all(start_state.current_state==goal):
        path.append(start_state.current_state)
        return True
    
    start_tile=np.argwhere(start_state.current_state==0)[0]
    starting_board=start_state.current_state

    directions =[
                    (-1,0), #left
                    (1,0),  #right
                    (0,-1), #down
                    (0,1),  #right
                ]
    
    for direction in directions:

        swapping_tile = tuple(map(lambda x, y: x + y, start_tile, direction))

        if not is_valid_move(swapping_tile,starting_board.shape[0]):
            continue

        landing_board=swap(starting_board,start_tile,swapping_tile)

        if node_already_present(graph,landing_board):
            #check if this step has been already considered
            continue

        new_node = Node(start_state,landing_board,start_state.depth+1)
        graph.append(new_node)

        if DFS_R(graph,path,new_node,goal,start):
            path.append(start_state)
            return True
        
    return False

def DFS(start_state:np.ndarray,goal:np.ndarray,start=0):
    '''
    This is a wrapper for the recursive function implementing the acutal DFS
    '''
    root = Node(None,start_state,0)
    graph = [root]
    path = []
    
    if not is_solvable(start_state) and start_state.shape[0]!=2:
        #cases not provided by the 'generate_board()' function must be tested
        return('This board cannot be solved!')

    if DFS_R(graph,path,root,goal,start):
        print(f'Quality:\t{len(path)}')
        print(f'Cost:\t{len(graph)}')
        for state in path:
            state.display()
            print('-->')
        print('Done!')
    
    else:
        print('Could not find any resolution to this problem')
    

### A*

In [5]:
PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

In [6]:
def available_actions(state: np.ndarray):
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [7]:
class Puzzle:
    def __init__(self, goal_state: np.ndarray):
        self.goal_state = goal_state

    def manhattan_distance(self, position: np.ndarray) -> int:
        distance = 0
        size = len(position)
        for i in range(size):
            for j in range(size):
                tile = position[i][j]
                if tile != 0:
                    target_row = (tile - 1) // size
                    target_col = (tile - 1) % size
                    distance += abs(i - target_row) + abs(j - target_col)
        return distance
    

def A_star(initial_state: np.ndarray, goal_state: np.ndarray):
    solution = Puzzle(goal_state)

    initial_state_bytes = initial_state.tobytes()

    def calculate_heuristic(state: np.ndarray):
        return solution.manhattan_distance(state)

    # Priority queue: (f_score, g_score, state, path)
    open_set = []
    heappush(open_set, (calculate_heuristic(initial_state), 0, initial_state_bytes, []))
    visited = set()

    evaluated_states = 0

    while open_set:
        # Extract the node with the lowest f_score
        f_score, g_score, current_bytes, path = heappop(open_set)
        current_state = np.frombuffer(current_bytes, dtype=initial_state.dtype).reshape((PUZZLE_DIM,PUZZLE_DIM))
        # Check if we've reached the goal state
        if np.all(current_state == goal_state):
            return True,path, evaluated_states

        # Add current state to visited
        visited.add(current_bytes)

        # Generate all possible moves
        for move in available_actions(current_state):
            evaluated_states += 1
            next_state = do_action(current_state, move)
            next_bytes = next_state.tobytes()
            
            
            if next_bytes in visited:
                continue
            

            # Update scores
            new_g_score = g_score + 1
            new_f_score = new_g_score + calculate_heuristic(next_state)

            # Add new state to open set
            heappush(open_set, (new_f_score, new_g_score, next_bytes, path + [move]))

    return False,'Puzzle unsolvable!',evaluated_states   #Puzzle is unsolvable


In [ ]:
##TRY DOUBLE DIRECTION A*

## Solving the puzzle

In [120]:
DIMENSIONS = 3

### Solving with DFS

In [139]:
game = generate_board(DIMENSIONS)
goal = [i for i in range(1,DIMENSIONS**2)]
goal.append(0)
goal = np.array(goal).reshape((DIMENSIONS,DIMENSIONS))
print(goal)
print('------')
print(game)
DFS(game,goal)

[[1 2 3]
 [4 5 6]
 [7 8 0]]
------
[[8 0 3]
 [5 2 1]
 [4 7 6]]
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursion
Recursi

RecursionError: maximum recursion depth exceeded

### Solving with BFS

In [ ]:
game = generate_board(DIMENSIONS)
goal = [i for i in range(1,DIMENSIONS**2)]
goal.append(0)
goal = np.array(goal).reshape((DIMENSIONS,DIMENSIONS))
print(goal)
print('------')
print(game)
BFS(game,goal)

### Solving with A*

In [8]:
RANDOMIZE_STEPS = 100_000

state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
ret_val = A_star(state,goal_state)
status, path, evaluated_states = A_star(state, goal_state)
print("Solution step by step:", path)
print("Number of states evaluated:", evaluated_states)
if status:
    print("Goodness of the solution: "  + str(len(path)) + " moves")
    print('Efficiency: ', round(len(path)/evaluated_states,2))